In [32]:
import pandas as pd
import pickle
import numpy as np 
import unicodedata
import re
import nltk
import time
import re
import unicodedata
import nltk
from tqdm import tqdm
import gensim

In [33]:
DATA_PATH = "/home/bennis/Bureau/5GMM/AI-Frameworks/DefiIA/defi-ia-insa-toulouse"

train_dfn = pd.read_json(DATA_PATH+"/train.json")
train_dfn.set_index('Id', inplace=True) 


In [45]:
DATA_PATH = "/home/bennis/Bureau/5GMM/AI-Frameworks/DefiIA/data" 

train_df = pd.read_csv(DATA_PATH+'/cleaned/cleaned_train.csv')
test_df = pd.read_csv(DATA_PATH+'/cleaned/cleaned_test.csv')
train_label = pd.read_csv(DATA_PATH+'/cleaned/train_label.csv')

In [59]:
class Word2Vec:

    def __init__(self, args):
        self.args = args

    def train(self):
        ts = time.time()
        model = gensim.models.Word2Vec(**self.args)
        te = time.time()
        return model, te-ts

    @staticmethod
    def get_features_mean(lines, model):
        features = [model[x] for x in lines if x in model]
        if features == []:
            fm = np.zeros(model.vector_size)
        else:
            fm = np.mean(features, axis=0)
        return fm

    @staticmethod
    def get_matrix_features_means(X, model):
        ts = time.time()
        X_embedded_ = list(map(lambda x: Word2Vec.get_features_mean(x, model), X))
        X_embedded = np.vstack(X_embedded_)
        te = time.time()
        return X_embedded, te-ts

In [46]:
array_token = [line.split(" ") for line in train_df["description_cleaned"].values]

In [42]:
features_dimension = 300
min_count = 1
window = 5
hs = 0
negative = 10

In [ ]:
we_sg = Word2Vec(args = dict(sentences = array_token, sg=1, hs=hs, 
                                  negative=negative, min_count=min_count, 
                                  size=features_dimension, window = window, iter=10))
model_sg, training_time_sg = we_sg.train()
print("Model Skip-gram trained in %.2f minutes"%(training_time_sg/60))

In [50]:
word_vectors = model_sg.wv

In [60]:
X_train, train_time = Word2Vec.get_matrix_features_means(array_token, model_sg)

<ipython-input-59-71b1a69040ec>:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  features = [model[x] for x in lines if x in model]
<ipython-input-59-71b1a69040ec>:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  features = [model[x] for x in lines if x in model]


In [62]:
len(X_train)

217193